In [ ]:
# Randomly sample files to read
import random
from pathlib import Path

in_dir = Path("data/librispeech-wav")
sample_size = 20

wav_paths = list(in_dir.rglob("*.wav"))
if sample_size < len(wav_paths):
    sampled_paths = random.sample(wav_paths, sample_size)
else:
    sampled_paths = wav_paths  

print(len(sampled_paths))

20


In [2]:
from torchaudio.functional import resample
import torch
import torchaudio
from tqdm import tqdm

hubert = torch.hub.load(
    "bshall/hubert:main",
    f"hubert_discrete",
    trust_repo=True,
)

acoustic_units = {}
filenames = {}
index = 0
for wav_path in tqdm(sampled_paths, desc="Encoding Audio Features"):
    wav, sr = torchaudio.load(wav_path)
    wav = resample(wav, sr, 16000)
    wav = wav.unsqueeze(0)

    with torch.inference_mode():
        units = hubert.units(wav)
    acoustic_units[wav_path.stem] = units.numpy()
    filenames[index] = wav_path.stem
    index += 1

Using cache found in /home/danel/.cache/torch/hub/bshall_hubert_main
Encoding Audio Features: 100%|██████████| 20/20 [00:06<00:00,  3.05it/s]
